In [ ]:
def get_saturn_moons_vectors(saturn_data_dict, epoch, units=True):
    """
    Get position and velocity vectors for Saturn's moons relative to Saturn.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        epoch (list): Julian date(s) or list of calendar dates to query.
        units (bool): Whether to use SI units (True) or AU (False). Default is True.
        
    Returns:
        dict: Nested dictionary with moon names as keys and updated data including position/velocity vectors.
    """
    results = {}
    center_id = '500@6'  # Center of the system is Saturn
    
    for moon, data in saturn_data_dict.items():
        target_id = data["ID"]
        # Query the Horizons database for each moon
        moon_data = Horizons(id=target_id, location=center_id, epochs=epoch, id_type=None)
        vectors = moon_data.vectors()
        
        # Extract position and velocity vectors
        if not units:
            position = {
                "x": float(vectors['x'][0]),
                "y": float(vectors['y'][0]),
                "z": float(vectors['z'][0])
            }
            velocity = {
                "vx": float(vectors['vx'][0]),
                "vy": float(vectors['vy'][0]),
                "vz": float(vectors['vz'][0])
            }
        else:
            position = {
                "x": float(vectors['x'][0] * AU),
                "y": float(vectors['y'][0] * AU),
                "z": float(vectors['z'][0] * AU)
            }
            velocity = {
                "vx": float(vectors['vx'][0] * AU / 24 / 60 / 60),
                "vy": float(vectors['vy'][0] * AU / 24 / 60 / 60),
                "vz": float(vectors['vz'][0] * AU / 24 / 60 / 60)
            }
        
        # Retain all existing keys and add new position/velocity vectors
        updated_data = data.copy()
        updated_data["r"] = position
        updated_data["v"] = velocity
        
        # Add to the results dictionary
        results[moon] = updated_data
    
    return results

In [ ]:
def get_saturn_moons_vectors_for_multiple_timesteps(saturn_data_dict, start_epoch, dt, number_of_timesteps, number_of_saved_points, units=True):
    """
    Get position and velocity vectors for Saturn's moons relative to Saturn over multiple timesteps.

    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        start_epoch (float): Start Julian date.
        dt (float): Time step spacing in seconds.
        number_of_timesteps (int): Total number of timesteps.
        number_of_saved_points (int): Number of points to save from the generated timesteps.
        units (bool): Whether to use SI units (True) or AU (False). Default is True.

    Returns:
        dict: Nested dictionary with moon names as keys and updated data including position/velocity vectors
              for specified timesteps indexed as r_i and v_i, as well as the corresponding time t_i.
    """
    results = {}
    center_id = '500@6'  # Center of the system is Saturn

    # Generate the array of timesteps and select saved points
    timesteps = np.linspace(0, dt * (number_of_timesteps - 1), number_of_timesteps) / 86400  # Convert seconds to days
    saved_indices = np.round(np.linspace(0, number_of_timesteps - 1, number_of_saved_points)).astype(int)
    saved_timesteps = start_epoch + timesteps[saved_indices]

    for moon, data in saturn_data_dict.items():
        target_id = data["ID"]
        updated_data = data.copy()

        try:
            # Query the Horizons database for the moon across all saved timesteps
            moon_data = Horizons(id=target_id, location=center_id, epochs=saved_timesteps, id_type=None)
            vectors = moon_data.vectors()

            # Extract and save position, velocity vectors, and time for each saved timestep
            for i, index in enumerate(saved_indices):
                try:
                    if not units:
                        position = {
                            f"r_{i}": {
                                "x": float(vectors['x'][i]),
                                "y": float(vectors['y'][i]),
                                "z": float(vectors['z'][i])
                            }
                        }
                        velocity = {
                            f"v_{i}": {
                                "vx": float(vectors['vx'][i]),
                                "vy": float(vectors['vy'][i]),
                                "vz": float(vectors['vz'][i])
                            }
                        }
                    else:
                        position = {
                            f"r_{i}": {
                                "x": float(vectors['x'][i] * AU),
                                "y": float(vectors['y'][i] * AU),
                                "z": float(vectors['z'][i] * AU)
                            }
                        }
                        velocity = {
                            f"v_{i}": {
                                "vx": float(vectors['vx'][i] * AU / 24 / 60 / 60),
                                "vy": float(vectors['vy'][i] * AU / 24 / 60 / 60),
                                "vz": float(vectors['vz'][i] * AU / 24 / 60 / 60)
                            }
                        }
                    
                    elapsed_seconds = saved_indices[i] * dt
                    time = {f"t_{i}": (saved_timesteps[i], elapsed_seconds)}

                    # Update the dictionary with indexed r_i, v_i, and t_i
                    updated_data.update(position)
                    updated_data.update(velocity)
                    updated_data.update(time)

                except Exception:
                    # Handle missing data for specific timesteps
                    updated_data[f"r_{i}"] = {"x": math.nan, "y": math.nan, "z": math.nan}
                    updated_data[f"v_{i}"] = {"vx": math.nan, "vy": math.nan, "vz": math.nan}
                    updated_data[f"t_{i}"] = math.nan

        except Exception:
            # Handle general errors in querying data for a moon
            for i in range(number_of_saved_points):
                updated_data[f"r_{i}"] = {"x": math.nan, "y": math.nan, "z": math.nan}
                updated_data[f"v_{i}"] = {"vx": math.nan, "vy": math.nan, "vz": math.nan}
                updated_data[f"t_{i}"] = math.nan

        # Add to the results dictionary
        results[moon] = updated_data

    return results


In [ ]:
def get_all_keys(saturn_data_dict):
    """
    Helper function to generate a list of all available keys in the subdictionaries.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        
    Returns:
        list: List of all keys found in the subdictionaries.
    """
    keys = set()
    for data in saturn_data_dict.values():
        keys.update(data.keys())
    return list(keys)

def convert_to_dataframe(saturn_data_dict, include_vectors=True, include_mass=True, include_time_independent=True):
    """
    Convert the saturn_data_dict to a pandas DataFrame and optionally unpack the r and v vectors, include mass, and other time-independent variables.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        include_vectors (bool): Whether to unpack the r and v vectors into separate columns. Default is True.
        include_mass (bool): Whether to include the Mass column. Default is True.
        include_time_independent (bool): Whether to include other time-independent variables. Default is True.
        
    Returns:
        DataFrame: DataFrame with moon names as objects and their data.
    """
    # Create a new dictionary to store the unpacked data
    unpacked_data = {}
    
    for moon, data in saturn_data_dict.items():
        unpacked_data[moon] = data.copy()
        
        if include_vectors:
            # Unpack r vector
            if "r" in data:
                unpacked_data[moon]["x"] = data["r"]["x"]
                unpacked_data[moon]["y"] = data["r"]["y"]
                unpacked_data[moon]["z"] = data["r"]["z"]
            
            # Unpack v vector
            if "v" in data:
                unpacked_data[moon]["vx"] = data["v"]["vx"]
                unpacked_data[moon]["vy"] = data["v"]["vy"]
                unpacked_data[moon]["vz"] = data["v"]["vz"]
        
        del unpacked_data[moon]["r"]
        del unpacked_data[moon]["v"]

        
        if not include_mass and "Mass" in unpacked_data[moon]:
            del unpacked_data[moon]["Mass"]
        
        if not include_time_independent:
            time_independent_keys = ["semi-major axis", "eccentricity", "inclination"]  # Add any other time-independent keys here
            for key in time_independent_keys:
                if key in unpacked_data[moon]:
                    del unpacked_data[moon][key]
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(unpacked_data, orient='index')
    
    # Reset index to have a column for the moon names
    df.reset_index(inplace=True)
    
    # Rename columns
    df.rename(columns={'index': 'Object'}, inplace=True)
    
    return df

def convert_to_cpp_vector(saturn_data_dict):
    """
    Convert the saturn_data_dict to a string representing a C++ vector of Body objects.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        
    Returns:
        str: String representing a C++ vector of Body objects.
    """
    # Convert the dictionary to a DataFrame
    df = convert_to_dataframe(saturn_data_dict, include_vectors=True, include_mass=True, include_time_independent=True)
    
    # Initialize the C++ vector string
    cpp_vector_str = "std::vector<Body> bodies = { \n"
    
    # Iterate through the DataFrame and construct the C++ vector string
    for index, row in df.iterrows():
        position = f"{{ {row['x']}, {row['y']}, {row['z']} }}"
        velocity = f"{{ {row['vx']}, {row['vy']}, {row['vz']} }}"
        cpp_vector_str += f'    {{"{row["Object"]}", {row["Mass"]}, {position}, {velocity}}},\n'
    
    # Close the vector string
    cpp_vector_str = cpp_vector_str.rstrip(",\n")  # Remove the trailing comma and newline
    cpp_vector_str += "\n};"
    
    return cpp_vector_str


In [ ]:
def get_all_keys(saturn_data_dict):
    """
    Helper function to generate a list of all available keys in the subdictionaries.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        
    Returns:
        list: List of all keys found in the subdictionaries.
    """
    keys = set()
    for data in saturn_data_dict.values():
        keys.update(data.keys())
        # Include keys for r_i and v_i vectors
        for key in data.keys():
            if re.match(r"r_\d+", key) or re.match(r"v_\d+", key):
                keys.add(key + "_x")
                keys.add(key + "_y")
                keys.add(key + "_z")
    return list(keys)

def extract_time_tuples(saturn_data_dict):
    """
    Extract the t_i tuples from the saturn_data_dict, assuming all moons have the same time steps.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        
    Returns:
        dict: Dictionary with t_i tuples as keys and their values.
    """
    time_tuples = {}

    # Get the time steps from the first moon (assuming all moons have the same time steps)
    first_moon = next(iter(saturn_data_dict))
    for key, value in saturn_data_dict[first_moon].items():
        if key.startswith("t_"):
            time_tuples[key] = value
    
    return time_tuples

def convert_to_dataframe(saturn_data_dict, include_vectors=True, include_mass=True, include_time_independent=True):
    """
    Convert the saturn_data_dict to a pandas DataFrame and optionally unpack the r and v vectors, include mass, and other time-independent variables.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        include_vectors (bool): Whether to unpack the r and v vectors into separate columns. Default is True.
        include_mass (bool): Whether to include the Mass column. Default is True.
        include_time_independent (bool): Whether to include other time-independent variables. Default is True.
        
    Returns:
        DataFrame: DataFrame with moon names as objects and their data.
    """
    # Create a new dictionary to store the processed data
    processed_data = {}
    
    for moon, data in saturn_data_dict.items():
        # Copy data for modification
        processed_data[moon] = data.copy()
        
        if include_vectors:
            # Unpack r_0, r_i vectors
            for key in list(data.keys()):
                if re.match(r"r_\d+", key):
                    vector_key = key
                    processed_data[moon][f"{vector_key}_x"] = data[vector_key]["x"]
                    processed_data[moon][f"{vector_key}_y"] = data[vector_key]["y"]
                    processed_data[moon][f"{vector_key}_z"] = data[vector_key]["z"]
                    del processed_data[moon][vector_key]
            
            # Unpack v_0, v_i vectors
            for key in list(data.keys()):
                if re.match(r"v_\d+", key):
                    vector_key = key
                    processed_data[moon][f"{vector_key}_vx"] = data[vector_key]["vx"]
                    processed_data[moon][f"{vector_key}_vy"] = data[vector_key]["vy"]
                    processed_data[moon][f"{vector_key}_vz"] = data[vector_key]["vz"]
                    del processed_data[moon][vector_key]
        
        else:
            # Remove r_0, v_0 keys if not including vectors
            vector_keys = [key for key in data.keys() if re.match(r"(r|v)_\d+", key)]
            for vector_key in vector_keys:
                del processed_data[moon][vector_key]

        # Remove mass if not included
        if not include_mass and "Mass" in processed_data[moon]:
            del processed_data[moon]["Mass"]
        
        # Remove time-independent variables if not included
        if not include_time_independent:
            time_independent_keys = ["semi-major axis", "eccentricity", "inclination"]  # Add more as needed
            for key in time_independent_keys:
                if key in processed_data[moon]:
                    del processed_data[moon][key]
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(processed_data, orient='index')
    
    # Reset index to have a column for the moon names
    df.reset_index(inplace=True)
    
    # Rename columns
    df.rename(columns={'index': 'Object'}, inplace=True)
    
    return df

def convert_to_cpp_vector(saturn_data_dict):
    """
    Convert the saturn_data_dict to a string representing a C++ vector of Body objects.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        
    Returns:
        str: String representing a C++ vector of Body objects.
    """
    # Convert the dictionary to a DataFrame
    df = convert_to_dataframe(saturn_data_dict, include_vectors=True, include_mass=True, include_time_independent=False)
    
    # Initialize the C++ vector string
    cpp_vector_str = "std::vector<Body> bodies = { \n"
    
    # Iterate through the DataFrame and construct the C++ vector string
    for index, row in df.iterrows():
        position = [f"{row[f'r_{i}_x']}, {row[f'r_{i}_y']}, {row[f'r_{i}_z']}" for i in range(len([col for col in df.columns if re.match(r"r_\d+_x", col)]))]
        velocity = [f"{row[f'v_{i}_vx']}, {row[f'v_{i}_vy']}, {row[f'v_{i}_vz']}" for i in range(len([col for col in df.columns if re.match(r"v_\d+_vx", col)]))]
        
        for pos, vel in zip(position, velocity):
            cpp_vector_str += f'    {{"{row["Object"]}", {row["Mass"]}, {{{pos}}}, {{{vel}}}}},\n'
    
    # Close the vector string
    cpp_vector_str = cpp_vector_str.rstrip(",\n")  # Remove the trailing comma and newline
    cpp_vector_str += "\n};"
    
    return cpp_vector_str

In [ ]:



def convert_to_dataframe(saturn_data_dict, include_vectors=True, include_mass=True, include_time_independent=True):
    """
    Convert the saturn_data_dict to a pandas DataFrame and optionally unpack the r and v vectors, include mass, and other time-independent variables.
    
    Parameters:
        saturn_data_dict (dict): Dictionary of moon names and their data.
        include_vectors (bool): Whether to unpack the r and v vectors into separate columns. Default is True.
        include_mass (bool): Whether to include the Mass column. Default is True.
        include_time_independent (bool): Whether to include other time-independent variables. Default is True.
        
    Returns:
        DataFrame: DataFrame with moon names as objects and their data.
    """
    # Create a new dictionary to store the unpacked data
    unpacked_data = {}
    
    for moon, data in saturn_data_dict.items():
        unpacked_data[moon] = data.copy()
        
        if include_vectors:
            # Unpack r_0, r_i vectors
            for key in list(data.keys()):
                if re.match(r"r_\d+", key):
                    vector_key = key
                    unpacked_data[moon][f"{vector_key}_x"] = data[vector_key]["x"]
                    unpacked_data[moon][f"{vector_key}_y"] = data[vector_key]["y"]
                    unpacked_data[moon][f"{vector_key}_z"] = data[vector_key]["z"]
                    del unpacked_data[moon][vector_key]
            
            # Unpack v_0, v_i vectors
            for key in list(data.keys()):
                if re.match(r"v_\d+", key):
                    vector_key = key
                    unpacked_data[moon][f"{vector_key}_vx"] = data[vector_key]["vx"]
                    unpacked_data[moon][f"{vector_key}_vy"] = data[vector_key]["vy"]
                    unpacked_data[moon][f"{vector_key}_vz"] = data[vector_key]["vz"]
                    del unpacked_data[moon][vector_key]
        
        if not include_mass and "Mass" in unpacked_data[moon]:
            del unpacked_data[moon]["Mass"]
        
        if not include_time_independent:
            time_independent_keys = ["semi-major axis", "eccentricity", "inclination"]  # Add any other time-independent keys here
            for key in time_independent_keys:
                if key in unpacked_data[moon]:
                    del unpacked_data[moon][key]
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(unpacked_data, orient='index')
    
    # Reset index to have a column for the moon names
    df.reset_index(inplace=True)
    
    # Rename columns
    df.rename(columns={'index': 'Object'}, inplace=True)
    
    return df


In [ ]:
def create_header(
    filepath,
    data_dict,
    start_data_folder,
    epoch,
    dt,
    timesteps,
    num_test_particles,
    saved_points_modularity,
    skipped_timesteps,
    inner_radius,
    outer_radius,
    include_particle_moon_collisions
):
    """
    Creates a header for the binary files.
    """
    moon_names = ", ".join(data_dict.keys())
    
    # Open the binary file in write mode
    with open(filepath, 'wb') as file:
        # Write the header information
        file.write(f"Moon Names: {moon_names}\n".encode())
        file.write(f"Start Data Folder: {start_data_folder}\n".encode())
        file.write(f"Epoch: {epoch}\n".encode())
        file.write(f"dt: {dt}\n".encode())
        file.write(f"Timesteps: {timesteps}\n".encode())
        file.write(f"Number of Test Particles: {num_test_particles}\n".encode())
        file.write(f"Saved Points Modularity: {saved_points_modularity}\n".encode())
        file.write(f"Skipped Timesteps: {skipped_timesteps}\n".encode())
        file.write(f"Inner Radius: {inner_radius}\n".encode())
        file.write(f"Outer Radius: {outer_radius}\n".encode())
        file.write(f"Include Particle-Moon Collisions: {include_particle_moon_collisions}\n".encode())
        file.write(b"End of Header\n")

    print(f"Header created and written to {filepath}")


In [ ]:
import os
from datetime import datetime

def create_header(
    filepath,
    data_dict,
    moon_count,
    initial_data_folder,
    epoch,
    dt,
    timesteps,
    num_test_particles,
    saved_points_modularity,
    skipped_timesteps,
    inner_radius,
    outer_radius,
    include_particle_moon_collisions
):
    """
    Creates a header for the binary files and prepends it to the existing file content.
    """
    moon_names = ", ".join(data_dict.keys())
    
    # Create a temporary file to write the header and existing content
    temp_filepath = filepath + '.tmp'
    
    with open(temp_filepath, 'wb') as temp_file:
        # Write the header information
        temp_file.write(f"Moon Names: {moon_names}\n".encode())
        temp_file.write(f"Moon Count: {moon_count}\n".encode())
        temp_file.write(f"Initial Data Folder: {initial_data_folder}\n".encode())
        temp_file.write(f"Epoch: {epoch}\n".encode())
        temp_file.write(f"dt: {dt}\n".encode())
        temp_file.write(f"Timesteps: {timesteps}\n".encode())
        temp_file.write(f"Number of Test Particles: {num_test_particles}\n".encode())
        temp_file.write(f"Saved Points Modularity: {saved_points_modularity}\n".encode())
        temp_file.write(f"Skipped Timesteps: {skipped_timesteps}\n".encode())
        temp_file.write(f"Inner Radius: {inner_radius}\n".encode())
        temp_file.write(f"Outer Radius: {outer_radius}\n".encode())
        temp_file.write(f"Include Particle-Moon Collisions: {include_particle_moon_collisions}\n".encode())
        # Write the end header marker
        temp_file.write(b"End of Header\n")

        # Append the content of the original file to the new file
        with open(filepath, 'rb') as original_file:
            temp_file.write(original_file.read())
    
    # Replace the original file with the new file
    import os
    os.replace(temp_filepath, filepath)

    print(f"Header created and prepended to {filepath}")


def read_header(filepath, output_type='string'):
    """
    Reads the header from the binary file until the 'End of Header' marker is found.
    By default, returns the header as a string. If output_type is 'dictionary', returns the header as a dictionary.
    """
    header = []

    # Open the binary file in read mode
    with open(filepath, 'rb') as file:
        while True:
            # Read line by line in binary mode
            line = file.readline()
            if b"End of Header" in line:
                break
            header.append(line)

    # Join the header lines into a single binary string and then decode
    header_string = b''.join(header).decode(errors='replace')  # Use 'replace' to handle undecodable bytes

    if output_type == 'dictionary':
        header_dict = {}
        # Split the header into lines
        lines = header_string.split('\n')
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                header_dict[key.strip()] = value.strip()
        return header_dict
    else:
        return header_string

def update_sublogfile(log_file_path, data_file_path, creation_date, source = "simulation"):
    """
    Update the log
    """
    # Creating data file_name
    creation_date_cleaned = creation_date.replace(":", "-").replace(" ", "_").replace(".", "-")

    if source == "simulation":
        data_filename = f"simulation {creation_date_cleaned}.bin"
    else:
        print("this part hasn't been finished yet")

    # Creating Log string
    Extra_info_string = f"File Name: {data_filename} \n"+f"Creation Date: {creation_date}"
    header_string = read_header(data_file_path)
    seperation_string = "---------------------------------------------------------------------------------------------------------"
    log_entry = "\n" + seperation_string + "\n" + Extra_info_string + "\n" + header_string + "\n" + seperation_string

    # Updating Log
    with open(log_file_path, 'a') as file:
        file.write(log_entry)
    
def run_simulation(
    data_dict,
    epoch,
    dt,
    timesteps,
    num_test_particles,
    saved_points_modularity,
    skipped_timesteps,
    inner_radius,
    outer_radius,
    include_particle_moon_collisions
):
    """
    Runs the N-body simulation with the given parameters and generates a binary output file with simulation data.

    Parameters:
    - data_dict (dict): Dictionary containing data of moons.
    - epoch (str): The starting epoch for the simulation.
    - dt (float): Time step for the simulation.
    - timesteps (int): Total number of timesteps to simulate.
    - num_test_particles (int): Number of test particles in the simulation.
    - saved_points_modularity (int): Frequency of saving data points.
    - skipped_timesteps (int): Number of initial timesteps to skip.
    - inner_radius (float): Inner radius for initializing test particles.
    - outer_radius (float): Outer radius for initializing test particles.
    - include_particle_moon_collisions (bool): Flag to include particle-moon collisions.

    Workflow:
    1. Gathers and rotates the data using `sms.get_horizons_data`.
    2. Creates a list of bodies for the simulation using `sms.generate_list_for_cpp_conversion`.
    3. Generates the output file name and path.
    4. Calculates the number of moons from the data dictionary.
    5. Runs the simulation using the `simulation.run_simulation` function.
    6. Creates a header for the binary output file using `create_header`.
    7. Updates the simulation log file with the simulation details.

    Returns:
    None
    """
    
    # Gathering and rotating data 
    rotated_dict, initial_data_folder = sms.get_horizons_data(data_dict,epoch,True,True)
    # initial_data_folder = "Temporary blank"

    # Creating the pybind11 list
    ls = sms.generate_list_for_cpp_conversion(rotated_dict)

    # Creating the output file name and path
    creation_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    creation_date_cleaned = creation_date.replace(":", "-").replace(" ", "_").replace(".", "-")
    output_filename = f"simulation {creation_date_cleaned}.bin"
    output_filepath = os.path.join(".","simulation_data",output_filename)

    # Calculating the Moon Count
    moon_count = len(data_dict.keys())

    # Running Simulation
    simulation.run_simulation(
    dt,
    timesteps,
    moon_count,
    num_test_particles,
    saved_points_modularity,
    skipped_timesteps,
    inner_radius,
    outer_radius,
    ls,
    output_filepath,
    include_particle_moon_collisions
    )

    # Create header
    create_header(
    output_filepath,
    data_dict,
    moon_count,
    initial_data_folder,
    epoch,
    dt,
    timesteps,
    num_test_particles,
    saved_points_modularity,
    skipped_timesteps,
    inner_radius,
    outer_radius,
    include_particle_moon_collisions
    )

    # Updating Log_file
    log_file_path = "./simulation_data/simulation_log_file"
    update_sublogfile(log_file_path, output_filepath, creation_date)

    

In [ ]:
import numpy as np
def read_binary_file(filepath):
    """
    Reads a binary file containing simulation data and extracts the positional information of celestial bodies.

    The function reads the header of the binary file to obtain metadata about the simulation, such as the number of celestial bodies (moon count). It then reads the binary data, starting from the end of the header, and reshapes it into an array containing positional and velocity information for each body.

    Parameters:
    filepath (str): The path to the binary file containing the simulation data.

    Returns:
    np.ndarray: A 3D NumPy array where the first dimension corresponds to the timesteps, the second dimension corresponds to the number of bodies, and the third dimension contains six elements representing the position (x, y, z) and velocity (vx, vy, vz) of each body.
    """
    info_dict = read_header(filepath, output_type="dictionary")
    length_header = len(read_header(filepath))+ len("End of Header\n")
    num_bodies = int(info_dict['Moon Count'])+int(info_dict['Number of Test Particles'])
    data = np.fromfile(filepath, dtype=np.float64, offset = length_header)
    positions = data.reshape(-1, num_bodies, 6)
    return positions


filepath = './simulation_data/simulation 2025-01-06_17-06-29.bin'

data = read_binary_file(filepath)
print(data.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
%matplotlib qt

class CelestialBody:
    """
    Represents a celestial body (e.g., moon or test particle) with attributes such as name, color, position, and trail options.

    Attributes:
        name (str): Name of the celestial body.
        color (str): Color used for plotting the body.
        pos (ndarray): 3D positional data (timesteps x 3).
        trail (bool): Whether the body should have a trailing line when animated.
    """
    def __init__(self, name, color, pos, trail=False):
        self.name = name
        self.color = color
        self.pos = pos  # Positional data as a 3D NumPy array (timesteps x 3)
        self.trail = trail


class Dataset:
    """
    Handles the simulation dataset, including reading the header, extracting positional data,
    and initializing celestial bodies for visualization.

    Attributes:
        filepath (str): Path to the binary simulation data file.
        header (dict): Header information extracted from the file.
        num_moons (int): Number of moons in the dataset.
        num_test_particles (int): Number of test particles in the dataset.
        positions (ndarray): 3D array of positions and velocities for all objects (timesteps x objects x 6).
        moons (list): List of CelestialBody objects representing moons.
        test_particles (list): List of CelestialBody objects representing test particles.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.header = self._read_header()
        self.num_moons = int(self.header['Moon Count'])
        self.num_test_particles = int(self.header['Number of Test Particles'])
        self.positions = self._read_binary_file()
        self.moons, self.test_particles = self._initialize_bodies()

    def _read_header(self):
        """
        Reads and parses the header section of the binary file.

        Returns:
            dict: Parsed header as a dictionary.
        """
        header = {}
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                key, value = line.decode('utf-8').strip().split(':', 1)
                header[key.strip()] = self._convert_value(value.strip())
        return header

    def _convert_value(self, value):
        """
        Converts header values to their appropriate data types.

        Args:
            value (str): Header value to be converted.

        Returns:
            int, float, or str: Converted value.
        """
        try:
            if '.' in value:
                return float(value)
            return int(value)
        except ValueError:
            return value

    def _read_binary_file(self):
        """
        Reads the binary data section of the file, skipping the header.

        Returns:
            ndarray: Reshaped positional and velocity data (timesteps x objects x 6).
        """
        length_header = len(self._read_header_raw()) + len("End of Header\n")
        data = np.fromfile(self.filepath, dtype=np.float64, offset=length_header)
        return data.reshape(-1, self.num_moons + self.num_test_particles, 6)

    def _read_header_raw(self):
        """
        Reads the raw header for calculating its length.

        Returns:
            bytes: Raw header data.
        """
        header = []
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                header.append(line)
        return b"".join(header)

    def _initialize_bodies(self):
        """
        Initializes celestial bodies (moons and test particles) with positional data.

        Returns:
            tuple: A list of moon CelestialBody objects and test particle CelestialBody objects.
        """
        moon_names = self.header['Moon Names'].split(', ')
        body_colors = ['yellow', 'red', 'chartreuse', 'lightblue', 'orange', 'brown', 'blue', 'pink', 'red', 'black',
                       'green', 'purple', 'cyan', 'magenta', 'gold', 'silver', 'lime', 'navy', 'maroon', 'crimson']
        moon_colors = body_colors[:self.num_moons]

        moons = [
            CelestialBody(moon_names[i], moon_colors[i], self.positions[:, i, :3], trail=True)
            for i in range(self.num_moons)
        ]

        test_particle_positions = self.positions[:, self.num_moons:, :3]
        test_particles = [
            CelestialBody(str(i), "navy", test_particle_positions[:, i, :], trail=False)
            for i in range(test_particle_positions.shape[1])
        ]

        return moons, test_particles

    def plot(self, coords=[0, 1], n_farthest_filter=10, big_traillength=100, small_traillength=3, interval=1):
        """
        Animates the simulation, displaying the positions and trails of celestial bodies.

        Args:
            coords (list): Coordinate indices to plot (e.g., [0, 1] for X-Y plane).
            n_farthest_filter (int): Number of moons to exclude from zoomed view based on distance.
            big_traillength (int): Length of the main trail for moons.
            small_traillength (int): Length of the zoomed trail for moons.
            interval (int): Time interval between animation frames.
        """
        # Create a figure with two subplots
        figure, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Set axis limits for the full view
        xmax, xmin = np.max([np.max(body.pos[:, coords[0]]) for body in self.moons]), np.min([np.min(body.pos[:, coords[0]]) for body in self.moons])
        ymax, ymin = np.max([np.max(body.pos[:, coords[1]]) for body in self.moons]), np.min([np.min(body.pos[:, coords[1]]) for body in self.moons])
        ax.set_xlim(xmin - 0.1 * abs(xmin), xmax + 0.1 * abs(xmax))
        ax.set_ylim(ymin - 0.1 * abs(ymin), ymax + 0.1 * abs(ymax))
        ax.set_aspect('equal', adjustable='box')

        # Filter moons for the zoomed-in view
        nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2) for body in self.moons])[:-n_farthest_filter]
        filtered_moons = [self.moons[i] for i in nth_largest_indices]
        ax2.set_xlim(-3e8, 3e8)
        ax2.set_ylim(-3e8, 3e8)
        ax2.set_aspect('equal', adjustable='box')

        # Initialize plot objects for moons and test particles
        moon_lines = {body: ax.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in self.moons}
        moon_markers = {}
        zoom_lines = {}
        zoom_markers = {}

        # Special marker size for Saturn
        large_marker_size = 15
        default_marker_size = 6

        for body in self.moons:
            marker_size = large_marker_size if body.name.lower() == "saturn" else default_marker_size
            moon_markers[body] = ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]
            if body in filtered_moons:
                zoom_lines[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0]
                zoom_markers[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]

        test_particle_line = ax2.plot([], [], ".", label="Test Particles", color="navy", markersize=1)[0]

        # Initialization function for the animation
        def init():
            for line in moon_lines.values():
                line.set_data([], [])
            for marker in moon_markers.values()#:
                marker.set_data([], [])
            for zoom_line in zoom_lines.values():
                zoom_line.set_data([], [])
            for zoom_marker in zoom_markers.values():
                zoom_marker.set_data([], [])
            test_particle_line.set_data([], [])
            return list(moon_lines.values()) + list(moon_markers.values()) + list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]

        # Update function for each frame
        def update(i):
            for body in self.moons:
                moon_lines[body].set_data(
                    body.pos[max(i - big_traillength, 0):i, coords[0]],
                    body.pos[max(i - big_traillength, 0):i, coords[1]]
                )
                moon_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            for body in filtered_moons:
                zoom_lines[body].set_data(
                    body.pos[max(i - small_traillength, 0):i, coords[0]],
                    body.pos[max(i - small_traillength, 0):i, coords[1]]
                )
                zoom_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            test_particle_line.set_data(
                self.positions[i, self.num_moons:, coords[0]],
                self.positions[i, self.num_moons:, coords[1]]
            )
            return list(moon_lines.values()) + list(moon_markers.values()) + list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]

        # Create the animation and assign it to an attribute to prevent garbage collection
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.positions.shape[0], interval),
            interval=interval,
            blit=False
        )

        # Set axis labels
        labels = {0: 'X (m)', 1: 'Y (m)', 2: 'Z (m)'}
        ax.set_xlabel(labels[coords[0]])
        ax.set_ylabel(labels[coords[1]])
        ax2.set_xlabel(labels[coords[0]])
        ax2.set_ylabel(labels[coords[1]])

        # Add a legend
        marker_handles = [moon_markers[body] for body in self.moons]
        figure.legend(handles=marker_handles, loc='center right', bbox_to_anchor=(0.98, 0.5), title="Moons")

        # Adjust layout for better readability
        plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0.15)

        plt.show()


# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-09_16-17-43.bin'
data = Dataset(filepath)
data.plot()


In [ ]:
import numpy as np
import os
from datetime import datetime

local_path_to_horizons_long_format_data = os.path.join(".", "SaturnModelDatabase", "horizons_long_format_data")

def write_binary_file_in_chunks(filename, positions, chunk_size=1000):
    with open(filename, 'ab') as f:
        for i in range(0, positions.shape[0], chunk_size):
            chunk = positions[i:i+chunk_size].flatten()
            chunk.tofile(f)

def create_header_horizons_long(
    filepath,
    data_dict,
    moon_count,
    initial_data_folder,
    epoch,
    dt,
    timesteps,
    saved_points_modularity,
    integrator = "Horizons"
):
    """
    Creates a header for the binary files and writes it to the specified file.
    """
    moon_names = ", ".join(data_dict.keys())
    
    with open(filepath, 'wb') as file:
        # Write the header information
        file.write(f"Moon Names: {moon_names}\n".encode())
        file.write(f"Moon Count: {moon_count}\n".encode())
        file.write(f"Initial Data Folder: {initial_data_folder}\n".encode())
        file.write(f"Epoch: {epoch}\n".encode())
        file.write(f"dt: {dt}\n".encode())
        file.write(f"Timesteps: {timesteps}\n".encode())
        file.write(f"Saved Points Modularity: {saved_points_modularity}\n".encode())
        file.write(f"Numerical Integrator: {integrator}\n".encode())
        # Write the end header marker
        file.write(b"End of Header\n")
    
    print(f"Header created and written to {filepath}")



def create_long_format_horizons_data(data_dict, epoch, dt, number_of_timesteps, number_of_saved_points):
    # Querying data:
    saturn_data_long = sms.get_saturn_moons_vectors(data_dict, epoch, dt, number_of_timesteps, number_of_saved_points, include_time=True, units=True)
    x, v = sms.rotate_data(saturn_data_long, epoch)
    # Restructure data:
    positions = np.transpose((np.concatenate((x, v), axis=1)), axes=[0, 2, 1])

    # Creating the output file name and path
    creation_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    creation_date_cleaned = creation_date.replace(":", "-").replace(" ", "_").replace(".", "-")
    output_filename = f"horizons_long {creation_date_cleaned}.bin"
    output_filepath = os.path.join(local_path_to_horizons_long_format_data, output_filename)

    #getting initial data folder
    initial_data_folder = sms.get_horizons_data(data_dict, epoch, return_dict=False, return_foldername = True)

    # Create header
    create_header_horizons_long(
        output_filepath, data_dict, moon_count=len(data_dict),
        initial_data_folder=initial_data_folder, epoch=epoch,
        dt=dt, timesteps=number_of_timesteps, saved_points_modularity=number_of_saved_points
    )

    # Saving data:
    write_binary_file_in_chunks(output_filepath, positions)

    # update log_file
    log_file_path = os.path.join(local_path_to_horizons_long_format_data,"horizons_long_format_log_file.txt")
    sms.update_sublogfile(log_file_path, output_filepath, creation_date, source = "horizons")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class CollisionAnalysis:
    def __init__(self, database):
        self.database = database
        self.pre_collision_positions = np.array([])  # Initialize as an empty NumPy array

    def collisions_count(self):
        # Calculate the distance of each particle from the origin
        distances = np.sqrt(np.sum(self.database.positions**2, axis=2))
        dt = self.database.header["dt"] * self.database.header["Saved Points Modularity"]

        # Identify particles that moved to distances greater than 1e11
        far_particles_mask = distances > 1e11

        # Identify positions one time step before moving to distances > 1e11
        pre_collision_mask = np.roll(far_particles_mask, 1, axis=0) & ~far_particles_mask

        # Store positions before they move to distances > 1e11
        self.pre_collision_positions = self.database.positions[pre_collision_mask]

        # Count the number of particles with a distance greater than 1e11
        far_particles_count = np.sum(far_particles_mask, axis=1)
        
        # Plot the count of far particles over time
        plt.figure(figsize=(10, 5))
        plt.plot(dt / 60 / 60 / 24 * np.arange(len(far_particles_count)), far_particles_count, label='Number of collisions')
        plt.xlabel('Time (days)')
        plt.ylabel('Count')
        plt.title('Count of Collisions')
        plt.legend()
        plt.show()

    def plot_radial_histogram(self, bins=30, r_min=0.7e8, r_max=1.4e8):
        if self.pre_collision_positions.size == 0:
            print("No pre-collision positions stored. Please run collisions_count() first.")
            return

        # Calculate the radial distances of pre-collision positions
        radial_distances = np.sqrt(np.sum(np.array(self.pre_collision_positions) ** 2, axis=1))

        # Create figure
        plt.figure(figsize=(10, 5))
        plt.hist(radial_distances, bins=bins, edgecolor='black', label="Collisions")

        # Close moons to highlight
        close_moons = ["Daphnis", "Mimas", "Janus", "Epimetheus", "Atlas", "Pandora", "Pan", "Prometheus", "Enceladus"]
        moon_data = {moon.name: moon for moon in self.database.moons}

        # Dictionary of moon radii in meters (if known)
        moon_radii = {
            "Daphnis": 4.6e3,  # meters
            "Mimas": 198.8e3,
            "Janus": 101.7e3,
            "Epimetheus": 64.9e3,
            "Atlas": 20.5e3,
            "Pandora": 52.2e3,
            "Pan": 17.2e3,
            "Prometheus": 68.8e3,
            "Enceladus": 252.3e3
        }

        patches = []  # For legend entries

        for moon in close_moons:
            if moon in moon_data:
                moon_obj = moon_data[moon]
                orbital_params, _ = moon_obj.calculate_orbital_elements()

                semi_major_axis = np.mean(orbital_params["semimajor_axis"])  # Ensure scalar
                eccentricity = np.mean(orbital_params["eccentricity"])  # Ensure scalar

                # Compute min and max radius from semi-major axis and eccentricity
                r_min_moon = semi_major_axis * (1 - eccentricity)
                r_max_moon = semi_major_axis * (1 + eccentricity)

                # Add the moon's physical radius (if known)
                if moon in moon_radii:
                    radius = moon_radii[moon]
                    r_min_moon -= radius
                    r_max_moon += radius

                # Plot shaded region
                patch = plt.axvspan(float(r_min_moon), float(r_max_moon), color=moon_obj.color, alpha=0.3, label=moon)
                patches.append(patch)

        # Format plot
        plt.xlim(r_min, r_max)
        plt.xlabel('Radial Distance')
        plt.ylabel('Frequency')
        plt.title('Radial Histogram of Collisions')

        # Place legend outside
        plt.legend(handles=patches, loc='upper left', bbox_to_anchor=(1, 1))
        plt.show()




